In [52]:
import pandas as pd
import numpy as np

In [53]:
# Read CSV file with raw data
df = pd.read_csv("/Users/rahultaduri/data-science-spring-2024/data-science-spring-2024/Project/Data/t9peq0oezmcdvtoo.csv")

/var/folders/xh/2wkw_sr97y5chj8h8jwwpd6r0000gn/T/ipykernel_35597/1495293352.py:2: DtypeWarning: Columns (23,26,32) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/Users/rahultaduri/data-science-spring-2024/data-science-spring-2024/Project/Data/t9peq0oezmcdvtoo.csv")


In [54]:
# Remove unnecessary columns
df = df[['date','exdate', 'last_date',
       'cp_flag', 'strike_price', 'best_bid', 'best_offer', 'volume',
       'open_interest', 'impl_volatility', 'delta', 'gamma', 'vega', 'theta',
       'optionid', 'contract_size', 'issuer']]

In [55]:
# Adding a new column with 'days to expiration
df[['date','exdate']] = df[['date','exdate']].apply(pd.to_datetime) #if conversion required
df['days_to_expiration'] = (df['exdate'] - df['date']).dt.days

In [56]:
# Group by ticker
grouped_df = df.groupby('issuer')
AMD_data_raw = grouped_df.get_group('ADVANCED MICRO DEVICES INC.')
NVDA_data_raw = grouped_df.get_group('NVIDIA CORP')
TSMC_data_raw = grouped_df.get_group('TAIWAN SEMICONDUCTOR MANUFAC')

In [62]:
# Load historical stock data
hist_AMD_price = pd.read_csv("/Users/rahultaduri/data-science-spring-2024/data-science-spring-2024/Project/Data/AMD_Stock.csv")
hist_NVDA_price = pd.read_csv("/Users/rahultaduri/data-science-spring-2024/data-science-spring-2024/Project/Data/NVDA_Stock.csv")
hist_TSMC_price = pd.read_csv("/Users/rahultaduri/data-science-spring-2024/data-science-spring-2024/Project/Data/TSMC_Stock.csv")

In [65]:
hist_AMD_price['Date'] = hist_AMD_price['Date'].apply(pd.to_datetime)

In [66]:
# Merge stock data to options dataframe, using only adjusted closing price for options purchase and expiry dates (AMD)
AMD_merge = AMD_data_raw.merge(hist_AMD_price[['Date','Adj Close']],how='left',left_on='date',right_on='Date')
AMD_merge = AMD_merge.merge(hist_AMD_price[['Date','Adj Close']],how='left',left_on='exdate',right_on='Date')
AMD_merge = AMD_merge.rename(columns = {'Adj Close_x':'Adj_Close_date','Adj Close_y':'Adj_Close_exdate'})
AMD_merge = AMD_merge.drop(columns = ['Date_x','Date_y'])

In [67]:
# Merge stock data to options dataframe, using only adjusted closing price for options purchase and expiry dates (NVDA)
NVDA_merge = NVDA_data_raw.merge(hist_AMD_price[['Date','Adj Close']],how='left',left_on='date',right_on='Date')
NVDA_merge = NVDA_merge.merge(hist_AMD_price[['Date','Adj Close']],how='left',left_on='exdate',right_on='Date')
NVDA_merge = NVDA_merge.rename(columns = {'Adj Close_x':'Adj_Close_date','Adj Close_y':'Adj_Close_exdate'})
NVDA_merge = NVDA_merge.drop(columns = ['Date_x','Date_y'])

In [68]:
# Merge stock data to options dataframe, using only adjusted closing price for options purchase and expiry dates (TSMC)
TSMC_merge = TSMC_data_raw.merge(hist_AMD_price[['Date','Adj Close']],how='left',left_on='date',right_on='Date')
TSMC_merge = TSMC_merge.merge(hist_AMD_price[['Date','Adj Close']],how='left',left_on='exdate',right_on='Date')
TSMC_merge = TSMC_merge.rename(columns = {'Adj Close_x':'Adj_Close_date','Adj Close_y':'Adj_Close_exdate'})
TSMC_merge = TSMC_merge.drop(columns = ['Date_x','Date_y'])

In [15]:
# Add historical volatility column (AMD)
"""AMD_merge['prct_change'] = AMD_merge['Adj_Close_date'].pct_change()
inter_list = AMD_merge.prct_change[1:]
hist_volatility = []
for i in range(1,len(inter_list)+1):
   hist_volatility.append(np.std(inter_list[:i])*np.sqrt(252))

hist_volatility = hist_volatility[::-1]
hist_volatility.append(float(0))
hist_volatility = hist_volatility[::-1]
AMD_merge["hist_volatility"] = hist_volatility"""

In [16]:
# Add historical volatility column (NVDA)
"""NVDA_merge['prct_change'] = NVDA_merge['Adj_Close_date'].pct_change()
inter_list = NVDA_merge.prct_change[1:]
hist_volatility = []
for i in range(1,len(inter_list)+1):
   hist_volatility.append(np.std(inter_list[:i])*np.sqrt(252))

hist_volatility = hist_volatility[::-1]
hist_volatility.append(float(0))
hist_volatility = hist_volatility[::-1]
NVDA_merge["hist_volatility"] = hist_volatility"""

In [17]:
# Add historical volatility column (TSMC)
"""TSMC_merge['prct_change'] = TSMC_merge['Adj_Close_date'].pct_change()
inter_list = TSMC_merge.prct_change[1:]
hist_volatility = []
for i in range(1,len(inter_list)+1):
   hist_volatility.append(np.std(inter_list[:i])*np.sqrt(252))

hist_volatility = hist_volatility[::-1]
hist_volatility.append(float(0))
hist_volatility = hist_volatility[::-1]
TSMC_merge["hist_volatility"] = hist_volatility"""

In [69]:
# Add risk free interest rate column (AMD)
rf_interest_rate = pd.read_csv("/Users/rahultaduri/data-science-spring-2024/data-science-spring-2024/Project/Data/TNX.csv",parse_dates=['Date'])
rf_interest_rate['date'] = rf_interest_rate.Date
rf_interest_rate = rf_interest_rate[['date','Open']]
AMD_merge = pd.merge(AMD_merge,rf_interest_rate,on=['date'],how='outer')
AMD_merge.rename(columns={'Open':'risk_free_rate'},inplace=True)

In [70]:
# Add risk free interest rate column (NVDA)
rf_interest_rate = pd.read_csv("/Users/rahultaduri/data-science-spring-2024/data-science-spring-2024/Project/Data/TNX.csv",parse_dates=['Date'])
rf_interest_rate['date'] = rf_interest_rate.Date
rf_interest_rate = rf_interest_rate[['date','Open']]
NVDA_merge = pd.merge(NVDA_merge,rf_interest_rate,on=['date'],how='outer')
NVDA_merge.rename(columns={'Open':'risk_free_rate'},inplace=True)

In [71]:
# Add risk free interest rate column (TSMC)
rf_interest_rate = pd.read_csv("/Users/rahultaduri/data-science-spring-2024/data-science-spring-2024/Project/Data/TNX.csv",parse_dates=['Date'])
rf_interest_rate['date'] = rf_interest_rate.Date
rf_interest_rate = rf_interest_rate[['date','Open']]
TSMC_merge = pd.merge(TSMC_merge,rf_interest_rate,on=['date'],how='outer')
TSMC_merge.rename(columns={'Open':'risk_free_rate'},inplace=True)

Add derived columns to dataframe:
1. Spread = Best Offer - Best Bid
2. Delta_UL = Underlying stock price change on option purchase and expiry dates
3. Delta UL & SP = Difference between option strike price and underlying stock price on option expiry date

In [72]:
# Implement formulas above for AMD
AMD_merge['Spread']=AMD_merge['best_offer']-AMD_merge['best_bid']
AMD_merge['Delta_Underlying']=AMD_merge['Adj_Close_exdate']-AMD_merge['Adj_Close_date']
AMD_merge['Delta_UL_and_SP']=AMD_merge['strike_price']-AMD_merge['Adj_Close_date']

In [73]:
# Implement formulas above for NVDA
NVDA_merge['Spread']=NVDA_merge['best_offer']-NVDA_merge['best_bid']
NVDA_merge['Delta_Underlying']=NVDA_merge['Adj_Close_exdate']-NVDA_merge['Adj_Close_date']
NVDA_merge['Delta_UL_and_SP']=NVDA_merge['strike_price']-NVDA_merge['Adj_Close_date']

In [74]:
# Implement formulas above for TSMC
TSMC_merge['Spread']= TSMC_merge['best_offer']- TSMC_merge['best_bid']
TSMC_merge['Delta_Underlying']= TSMC_merge['Adj_Close_exdate']- TSMC_merge['Adj_Close_date']
TSMC_merge['Delta_UL_and_SP']= TSMC_merge['strike_price']- TSMC_merge['Adj_Close_date']

In [75]:
# Adding a new column - sigma
AMD_merge['sigma_Adj_close'] = AMD_merge.Adj_Close_date*AMD_merge.impl_volatility*np.sqrt(AMD_merge.days_to_expiration)/365
NVDA_merge['sigma_Adj_close'] = NVDA_merge.Adj_Close_date*NVDA_merge.impl_volatility*np.sqrt(NVDA_merge.days_to_expiration)/365
TSMC_merge['sigma_Adj_close'] = TSMC_merge.Adj_Close_date*TSMC_merge.impl_volatility*np.sqrt(TSMC_merge.days_to_expiration)/365

In [76]:
# AMD puts and call
AMD_Puts = AMD_merge[AMD_merge['cp_flag']== 'P']
AMD_Calls = AMD_merge[AMD_merge['cp_flag']== 'C']

In [78]:
# NVDA puts and call
NVDA_Puts = NVDA_merge[NVDA_merge['cp_flag']== 'P']
NVDA_Calls = NVDA_merge[NVDA_merge['cp_flag']== 'C']

In [79]:
# NVDA puts and call
TSMC_Puts = TSMC_merge[TSMC_merge['cp_flag']== 'P']
TSMC_Calls = TSMC_merge[TSMC_merge['cp_flag']== 'C']

In [83]:
AMD_Puts.columns


Index(['date', 'exdate', 'last_date', 'cp_flag', 'strike_price', 'best_bid',
       'best_offer', 'volume', 'open_interest', 'impl_volatility', 'delta',
       'gamma', 'vega', 'theta', 'optionid', 'contract_size', 'issuer',
       'days_to_expiration', 'Adj_Close_date', 'Adj_Close_exdate',
       'risk_free_rate', 'Spread', 'Delta_Underlying', 'Delta_UL_and_SP',
       'sigma_Adj_close'],
      dtype='object')